In [3]:
# import required libs

import numpy as np
import pandas as pd
import sys
sys.path.insert(1, 'config.py')
from config import *
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
from time import sleep

In [2]:
# Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="458691ea408c4bdf93cfa4cd69a7ccbf", client_secret="2c00181db1bb4f82b62ab501cac7c5ab"))


In [3]:
# read the csvs into dataframes

hot_100 = pd.read_csv("hot_100.csv")
not_hot = pd.read_csv("not_hot_100.csv")


display(hot_100.head())
display(not_hot.head())

,Song,Artist
0,Paint The Town Red,Doja Cat
1,Snooze,SZA
2,Fast Car,Luke Combs
3,Cruel Summer,Taylor Swift
4,I Remember Everything,Zach Bryan Featuring Kacey Musgraves


,Unnamed: 0.1,Unnamed: 0,track_id,Artist,album_name,Song,popularity,duration_ms,explicit,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,113186,113186,6KwkVtXm8OUp2XffN5k7lY,Hillsong Worship,No Other Name,No Other Name,50,440247,False,0.369,...,-6.984,1,0.0304,0.00511,0.000000,0.176,0.0466,148.014,4,world-music
1,42819,42819,2dp5I5MJ8bQQHDoFaNRFtX,Internal Rot,Grieving Birth,Failed Organum,11,93933,False,0.171,...,-3.586,1,0.1180,0.00521,0.801000,0.420,0.0294,122.223,4,grindcore
2,59311,59311,5avw06usmFkFrPjX8NxC40,Zhoobin Askarieh;Ali Sasha,Noise A Noise 20.4-1,"Save the Trees, Pt. 1",0,213578,False,0.173,...,-10.071,0,0.1440,0.61300,0.001910,0.195,0.0887,75.564,3,iranian
3,91368,91368,75hT0hvlESnDJstem0JgyR,Bryan Adams,All I Want For Christmas Is You,Merry Christmas,0,151387,False,0.683,...,-5.598,1,0.0279,0.40600,0.000197,0.111,0.5980,109.991,3,rock
4,61000,61000,4bY2oZGA5Br3pTE1Jd1IfY,Nogizaka46,バレッタ TypeD,月の大きさ,57,236293,False,0.555,...,-3.294,0,0.0481,0.48400,0.000000,0.266,0.8130,92.487,4,j-idol


In [4]:
# check that the duplicates have been deleted
print(not_hot.shape)

(2997, 22)


In [5]:
#drop cols apart from Song and Artist

not_hot = not_hot[['Song', 'Artist']]
display(not_hot.head())

,Song,Artist
0,No Other Name,Hillsong Worship
1,Failed Organum,Internal Rot
2,"Save the Trees, Pt. 1",Zhoobin Askarieh;Ali Sasha
3,Merry Christmas,Bryan Adams
4,月の大きさ,Nogizaka46


In [6]:
# check if the seach on Spotify works

song = sp.search(q="tracks: Distortion Sleep artist: Soilwork", limit=1) 
song['tracks']['items'][0]['id']

'3yPQrLnNIthn4I5wQ51X26'

In [7]:
#function to search for songs

def search_song(title, artist, limit=5):
    query = "tracks: "+title+" artist: "+artist
    try:
        result = sp.search(query, limit=limit)
        song_id = result['tracks']['items'][0]['id']
        return song_id
    except:
        return "not found"



In [8]:
# use the function to return a song ID
search_song("On My Mama", "Victoria Monet")

'1o8Z7GD1CeOaVBEyuzu4HO'

In [9]:
# add a new column 'Spotify_ID' to the hot_100 DataFrame
hot_100['Spotify_ID'] = hot_100.apply(lambda row: search_song(row['Song'], row['Artist']), axis=1)

# filter out rows with "not found" as Spotify_ID
hot_100 = hot_100[hot_100['Spotify_ID'] != "not found"]

# reset the index after filtering, you can do so
hot_100.reset_index(drop=True, inplace=True)
display(hot_100)

,Song,Artist,Spotify_ID
0,Paint The Town Red,Doja Cat,2IGMVunIBsBLtEQyoI1Mu7
1,Snooze,SZA,4iZ4pt7kvcaH6Yo8UoZ4s2
2,Fast Car,Luke Combs,1Lo0QY9cvc8sUB2vnIOxDT
3,Cruel Summer,Taylor Swift,2EGaDf0cPX789H3LNeB03D
4,I Remember Everything,Zach Bryan Featuring Kacey Musgraves,4KULAymBBJcPRpk1yO4dOG
...,...,...,...
94,Standing Room Only,Tim McGraw,6BOP0cv1eeXcvi1oE8bDVZ
95,Checkmate,Rod Wave,6VdBDm20nkyk6A29P785aJ
96,Can't Have Mine,Dylan Scott,3jfywRZH6cB2iLyKqo4EZd
97,On My Mama,Victoria Monet,1o8Z7GD1CeOaVBEyuzu4HO


In [10]:
# add a new column 'Spotify_ID' to the not_hot dataframe
not_hot['Spotify_ID'] = not_hot.apply(lambda row: search_song(row['Song'], row['Artist']), axis=1)

# filter out rows with "not found" as Spotify_ID
not_hot = not_hot[not_hot['Spotify_ID'] != "not found"]

# reset the index after filtering, you can do so
not_hot.reset_index(drop=True, inplace=True)
display(not_hot)

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'tracks: Love Mashup artist: Shekhar Ravjiani;Shilpa Rao;Aakanksha Sharma;Ajay Gogavale;Amit Trivedi;Arijit Singh;Arko;Arnab Dutta;Ash King;Dev Negi;Farhan Saeed;Harshdeep Kaur;Neha Kakkar;Palak Muchhal;Shreya Ghoshal;Sunidhi Chauhan;Tony Kakkar;Yasser Desai;Ajay-Atul;Amjad Nadeem;Chirantan Bhat;Harish Sagane;JAM8;Pritam;Rochak Kohli;Shivam;Vishal-Shekhar;Dj Raahul Pai;Atif Aslam;Jyotica Tangri', 'limit': 5, 'offset': 0, 'type': 'track', 'market': None} returned 400 due to Bad request.
HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'tracks: Almost Like Praying (feat. Artists for Puerto Rico) artist: Lin-Manuel Miranda;Alex Lacamoire;Ana Villafañe;Anthony Ramos;Artists for Puerto Rico;Camila Cabello;Dessa;Ednita Nazario;Fat Joe;Gilberto Santa Rosa;Gina Rodriguez;Gloria Estefan;Jennifer Lopez;Joell Ortiz;John Leguizamo;Juan Luis Guerra 4.40;Luis Fonsi;Marc Anthony;Pj Sin Suela;Pedro Cap

,Song,Artist,Spotify_ID
0,No Other Name,Hillsong Worship,51AbDalUWkxCB1N0NZ3nU1
1,Failed Organum,Internal Rot,2dp5I5MJ8bQQHDoFaNRFtX
2,"Save the Trees, Pt. 1",Zhoobin Askarieh;Ali Sasha,7okbmgA8lRBGl5limZ7LFM
3,Merry Christmas,Bryan Adams,6B8tOwbbbvRkGBrFw2PJB1
4,月の大きさ,Nogizaka46,4bY2oZGA5Br3pTE1Jd1IfY
...,...,...,...
2982,Distortion Sleep,Soilwork,3yPQrLnNIthn4I5wQ51X26
2983,Proibida Pra Mim,Zeca Baleiro,5Te6AchmV1n93VD6DUDN3X
2984,"相愛很難 - 電影""男人四十""歌曲",Jacky Cheung;Anita Mui,09PZUuqCdaZi7KEIXdmoEQ
2985,The Girl From Ipanema,Antônio Carlos Jobim;Frank Sinatra,5N6pQ2vYtd3Rb9w7LC8PZ9


In [17]:
display(not_hot)

,Song,Artist,Spotify_ID,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,No Other Name,Hillsong Worship,51AbDalUWkxCB1N0NZ3nU1,0.358,0.613,7.0,-6.452,1.0,0.0308,0.005640,...,0.1460,0.0481,148.002,audio_features,51AbDalUWkxCB1N0NZ3nU1,spotify:track:51AbDalUWkxCB1N0NZ3nU1,https://api.spotify.com/v1/tracks/51AbDalUWkxC...,https://api.spotify.com/v1/audio-analysis/51Ab...,440253.0,4.0
1,Failed Organum,Internal Rot,2dp5I5MJ8bQQHDoFaNRFtX,0.171,0.997,7.0,-3.586,1.0,0.1180,0.005210,...,0.4200,0.0294,122.223,audio_features,2dp5I5MJ8bQQHDoFaNRFtX,spotify:track:2dp5I5MJ8bQQHDoFaNRFtX,https://api.spotify.com/v1/tracks/2dp5I5MJ8bQQ...,https://api.spotify.com/v1/audio-analysis/2dp5...,93933.0,4.0
2,"Save the Trees, Pt. 1",Zhoobin Askarieh;Ali Sasha,7okbmgA8lRBGl5limZ7LFM,0.528,0.234,3.0,-15.784,1.0,0.0283,0.641000,...,0.1830,0.1910,95.911,audio_features,7okbmgA8lRBGl5limZ7LFM,spotify:track:7okbmgA8lRBGl5limZ7LFM,https://api.spotify.com/v1/tracks/7okbmgA8lRBG...,https://api.spotify.com/v1/audio-analysis/7okb...,218733.0,4.0
3,Merry Christmas,Bryan Adams,6B8tOwbbbvRkGBrFw2PJB1,0.497,0.463,6.0,-7.465,1.0,0.0272,0.714000,...,0.1460,0.4680,73.318,audio_features,6B8tOwbbbvRkGBrFw2PJB1,spotify:track:6B8tOwbbbvRkGBrFw2PJB1,https://api.spotify.com/v1/tracks/6B8tOwbbbvRk...,https://api.spotify.com/v1/audio-analysis/6B8t...,150697.0,4.0
4,月の大きさ,Nogizaka46,4bY2oZGA5Br3pTE1Jd1IfY,0.555,0.941,9.0,-3.294,0.0,0.0481,0.484000,...,0.2660,0.8130,92.487,audio_features,4bY2oZGA5Br3pTE1Jd1IfY,spotify:track:4bY2oZGA5Br3pTE1Jd1IfY,https://api.spotify.com/v1/tracks/4bY2oZGA5Br3...,https://api.spotify.com/v1/audio-analysis/4bY2...,236293.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2846,Extraño - Versión Acústica,Miranda!,6az5WAcx20hzYkvSDOkj7n,0.731,0.567,7.0,-6.862,1.0,0.0376,0.779000,...,0.0726,0.8180,118.001,audio_features,6az5WAcx20hzYkvSDOkj7n,spotify:track:6az5WAcx20hzYkvSDOkj7n,https://api.spotify.com/v1/tracks/6az5WAcx20hz...,https://api.spotify.com/v1/audio-analysis/6az5...,232672.0,4.0
2847,Distortion Sleep,Soilwork,3yPQrLnNIthn4I5wQ51X26,0.368,0.949,2.0,-6.178,1.0,0.1410,0.000078,...,0.0341,0.3470,160.270,audio_features,3yPQrLnNIthn4I5wQ51X26,spotify:track:3yPQrLnNIthn4I5wQ51X26,https://api.spotify.com/v1/tracks/3yPQrLnNIthn...,https://api.spotify.com/v1/audio-analysis/3yPQ...,225933.0,4.0
2848,Proibida Pra Mim,Zeca Baleiro,5Te6AchmV1n93VD6DUDN3X,0.650,0.361,7.0,-10.614,1.0,0.0328,0.722000,...,0.0970,0.5400,132.839,audio_features,5Te6AchmV1n93VD6DUDN3X,spotify:track:5Te6AchmV1n93VD6DUDN3X,https://api.spotify.com/v1/tracks/5Te6AchmV1n9...,https://api.spotify.com/v1/audio-analysis/5Te6...,195000.0,4.0
2849,"相愛很難 - 電影""男人四十""歌曲",Jacky Cheung;Anita Mui,09PZUuqCdaZi7KEIXdmoEQ,0.448,0.730,0.0,-4.000,1.0,0.0385,0.526000,...,0.1230,0.4500,134.037,audio_features,09PZUuqCdaZi7KEIXdmoEQ,spotify:track:09PZUuqCdaZi7KEIXdmoEQ,https://api.spotify.com/v1/tracks/09PZUuqCdaZi...,https://api.spotify.com/v1/audio-analysis/09PZ...,224893.0,4.0


In [12]:
def get_audio_features(list_of_song_ids):
  
    """Collects audio features for a list of song IDs.

    Input:
    list_of_song_ids: A list of song IDs.

    Returns:
    pandas DataFrame containing the audio features for the IDs.
    """
    
    # split the list of song IDs into chunks of 50 to avoid hitting the rate limit.
    
    chunks = [list_of_song_ids[i:i + 50] for i in range(0, len(list_of_song_ids), 50)]

    # get audio features
    
    all_features = []
    for chunk in chunks:
        features_chunk = sp.audio_features(chunk)
        if features_chunk is not None:  # check if the result is not None.
            valid_features = [f for f in features_chunk if f is not None]
            all_features.extend(valid_features)
        else:
            print("Failed to fetch features for chunk:", chunk)

    # convert the list of dictionaries to df
    
    df_audio_features = pd.DataFrame(all_features)
    return df_audio_features

In [13]:
# get audio features for the hot songs

df_hot_audio_features = get_audio_features(hot_100['Spotify_ID'].tolist())

# merge frames

hot_100 = pd.merge(hot_100, df_hot_audio_features, left_on='Spotify_ID', right_on='id', how='left')

#drop duplicates

hot_100.drop_duplicates(subset=['Spotify_ID'], inplace=True)

# reindex
hot_100.reset_index(drop=True, inplace=True)

display(hot_100)


,Song,Artist,Spotify_ID,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Paint The Town Red,Doja Cat,2IGMVunIBsBLtEQyoI1Mu7,0.868,0.538,5,-8.603,1,0.1740,0.26900,...,0.0901,0.732,99.968,audio_features,2IGMVunIBsBLtEQyoI1Mu7,spotify:track:2IGMVunIBsBLtEQyoI1Mu7,https://api.spotify.com/v1/tracks/2IGMVunIBsBL...,https://api.spotify.com/v1/audio-analysis/2IGM...,231750,4
1,Snooze,SZA,4iZ4pt7kvcaH6Yo8UoZ4s2,0.559,0.551,5,-7.231,1,0.1320,0.14100,...,0.1100,0.392,143.008,audio_features,4iZ4pt7kvcaH6Yo8UoZ4s2,spotify:track:4iZ4pt7kvcaH6Yo8UoZ4s2,https://api.spotify.com/v1/tracks/4iZ4pt7kvcaH...,https://api.spotify.com/v1/audio-analysis/4iZ4...,201800,4
2,Fast Car,Luke Combs,1Lo0QY9cvc8sUB2vnIOxDT,0.712,0.603,8,-5.520,1,0.0262,0.18600,...,0.1150,0.670,97.994,audio_features,1Lo0QY9cvc8sUB2vnIOxDT,spotify:track:1Lo0QY9cvc8sUB2vnIOxDT,https://api.spotify.com/v1/tracks/1Lo0QY9cvc8s...,https://api.spotify.com/v1/audio-analysis/1Lo0...,265493,4
3,Cruel Summer,Taylor Swift,2EGaDf0cPX789H3LNeB03D,0.666,0.871,1,-6.017,0,0.0272,0.27000,...,0.2630,0.936,108.501,audio_features,2EGaDf0cPX789H3LNeB03D,spotify:track:2EGaDf0cPX789H3LNeB03D,https://api.spotify.com/v1/tracks/2EGaDf0cPX78...,https://api.spotify.com/v1/audio-analysis/2EGa...,215326,4
4,I Remember Everything,Zach Bryan Featuring Kacey Musgraves,4KULAymBBJcPRpk1yO4dOG,0.429,0.453,0,-7.746,1,0.0459,0.55400,...,0.1020,0.155,77.639,audio_features,4KULAymBBJcPRpk1yO4dOG,spotify:track:4KULAymBBJcPRpk1yO4dOG,https://api.spotify.com/v1/tracks/4KULAymBBJcP...,https://api.spotify.com/v1/audio-analysis/4KUL...,227196,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Standing Room Only,Tim McGraw,6BOP0cv1eeXcvi1oE8bDVZ,0.475,0.789,1,-6.124,1,0.0380,0.17000,...,0.1470,0.485,155.227,audio_features,6BOP0cv1eeXcvi1oE8bDVZ,spotify:track:6BOP0cv1eeXcvi1oE8bDVZ,https://api.spotify.com/v1/tracks/6BOP0cv1eeXc...,https://api.spotify.com/v1/audio-analysis/6BOP...,226091,4
92,Checkmate,Rod Wave,6VdBDm20nkyk6A29P785aJ,0.731,0.622,5,-8.237,1,0.0979,0.17600,...,0.1530,0.445,82.494,audio_features,6VdBDm20nkyk6A29P785aJ,spotify:track:6VdBDm20nkyk6A29P785aJ,https://api.spotify.com/v1/tracks/6VdBDm20nkyk...,https://api.spotify.com/v1/audio-analysis/6VdB...,198034,4
93,Can't Have Mine,Dylan Scott,3jfywRZH6cB2iLyKqo4EZd,0.663,0.502,5,-7.070,1,0.0285,0.79100,...,0.1050,0.371,143.969,audio_features,3jfywRZH6cB2iLyKqo4EZd,spotify:track:3jfywRZH6cB2iLyKqo4EZd,https://api.spotify.com/v1/tracks/3jfywRZH6cB2...,https://api.spotify.com/v1/audio-analysis/3jfy...,183087,4
94,On My Mama,Victoria Monet,1o8Z7GD1CeOaVBEyuzu4HO,0.481,0.539,4,-7.865,0,0.1080,0.55100,...,0.1200,0.621,153.277,audio_features,1o8Z7GD1CeOaVBEyuzu4HO,spotify:track:1o8Z7GD1CeOaVBEyuzu4HO,https://api.spotify.com/v1/tracks/1o8Z7GD1CeOa...,https://api.spotify.com/v1/audio-analysis/1o8Z...,186678,4


In [14]:
# get audio features for the not hot songs

df_not_hot_audio_features = get_audio_features(not_hot['Spotify_ID'].tolist())

# merge frames

not_hot = pd.merge(not_hot, df_not_hot_audio_features, left_on='Spotify_ID', right_on='id', how='left')

# drop duplicates
not_hot.drop_duplicates(subset=['Spotify_ID'], inplace=True)

# reindex
not_hot.reset_index(drop=True, inplace=True)

display(not_hot)

,Song,Artist,Spotify_ID,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,No Other Name,Hillsong Worship,51AbDalUWkxCB1N0NZ3nU1,0.358,0.613,7.0,-6.452,1.0,0.0308,0.005640,...,0.1460,0.0481,148.002,audio_features,51AbDalUWkxCB1N0NZ3nU1,spotify:track:51AbDalUWkxCB1N0NZ3nU1,https://api.spotify.com/v1/tracks/51AbDalUWkxC...,https://api.spotify.com/v1/audio-analysis/51Ab...,440253.0,4.0
1,Failed Organum,Internal Rot,2dp5I5MJ8bQQHDoFaNRFtX,0.171,0.997,7.0,-3.586,1.0,0.1180,0.005210,...,0.4200,0.0294,122.223,audio_features,2dp5I5MJ8bQQHDoFaNRFtX,spotify:track:2dp5I5MJ8bQQHDoFaNRFtX,https://api.spotify.com/v1/tracks/2dp5I5MJ8bQQ...,https://api.spotify.com/v1/audio-analysis/2dp5...,93933.0,4.0
2,"Save the Trees, Pt. 1",Zhoobin Askarieh;Ali Sasha,7okbmgA8lRBGl5limZ7LFM,0.528,0.234,3.0,-15.784,1.0,0.0283,0.641000,...,0.1830,0.1910,95.911,audio_features,7okbmgA8lRBGl5limZ7LFM,spotify:track:7okbmgA8lRBGl5limZ7LFM,https://api.spotify.com/v1/tracks/7okbmgA8lRBG...,https://api.spotify.com/v1/audio-analysis/7okb...,218733.0,4.0
3,Merry Christmas,Bryan Adams,6B8tOwbbbvRkGBrFw2PJB1,0.497,0.463,6.0,-7.465,1.0,0.0272,0.714000,...,0.1460,0.4680,73.318,audio_features,6B8tOwbbbvRkGBrFw2PJB1,spotify:track:6B8tOwbbbvRkGBrFw2PJB1,https://api.spotify.com/v1/tracks/6B8tOwbbbvRk...,https://api.spotify.com/v1/audio-analysis/6B8t...,150697.0,4.0
4,月の大きさ,Nogizaka46,4bY2oZGA5Br3pTE1Jd1IfY,0.555,0.941,9.0,-3.294,0.0,0.0481,0.484000,...,0.2660,0.8130,92.487,audio_features,4bY2oZGA5Br3pTE1Jd1IfY,spotify:track:4bY2oZGA5Br3pTE1Jd1IfY,https://api.spotify.com/v1/tracks/4bY2oZGA5Br3...,https://api.spotify.com/v1/audio-analysis/4bY2...,236293.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2846,Extraño - Versión Acústica,Miranda!,6az5WAcx20hzYkvSDOkj7n,0.731,0.567,7.0,-6.862,1.0,0.0376,0.779000,...,0.0726,0.8180,118.001,audio_features,6az5WAcx20hzYkvSDOkj7n,spotify:track:6az5WAcx20hzYkvSDOkj7n,https://api.spotify.com/v1/tracks/6az5WAcx20hz...,https://api.spotify.com/v1/audio-analysis/6az5...,232672.0,4.0
2847,Distortion Sleep,Soilwork,3yPQrLnNIthn4I5wQ51X26,0.368,0.949,2.0,-6.178,1.0,0.1410,0.000078,...,0.0341,0.3470,160.270,audio_features,3yPQrLnNIthn4I5wQ51X26,spotify:track:3yPQrLnNIthn4I5wQ51X26,https://api.spotify.com/v1/tracks/3yPQrLnNIthn...,https://api.spotify.com/v1/audio-analysis/3yPQ...,225933.0,4.0
2848,Proibida Pra Mim,Zeca Baleiro,5Te6AchmV1n93VD6DUDN3X,0.650,0.361,7.0,-10.614,1.0,0.0328,0.722000,...,0.0970,0.5400,132.839,audio_features,5Te6AchmV1n93VD6DUDN3X,spotify:track:5Te6AchmV1n93VD6DUDN3X,https://api.spotify.com/v1/tracks/5Te6AchmV1n9...,https://api.spotify.com/v1/audio-analysis/5Te6...,195000.0,4.0
2849,"相愛很難 - 電影""男人四十""歌曲",Jacky Cheung;Anita Mui,09PZUuqCdaZi7KEIXdmoEQ,0.448,0.730,0.0,-4.000,1.0,0.0385,0.526000,...,0.1230,0.4500,134.037,audio_features,09PZUuqCdaZi7KEIXdmoEQ,spotify:track:09PZUuqCdaZi7KEIXdmoEQ,https://api.spotify.com/v1/tracks/09PZUuqCdaZi...,https://api.spotify.com/v1/audio-analysis/09PZ...,224893.0,4.0


In [18]:
hot_100_featured = hot_100.copy()

hot_100_featured.to_csv('extended_hot_100.csv', index=False)

In [19]:
hot_100_featured = not_hot.copy()

not_hot_featured.to_csv('extended_not_hot.csv', index=False)